In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import pandas as pd
import dask as d
import os
import json
from glob import glob

In [ ]:
from fl_tissue_model_tools import data_prep, dev_config, models, defs, zstacks
import fl_tissue_model_tools.preprocessing as prep
import fl_tissue_model_tools.zstacks as zs

In [ ]:
dirs = dev_config.get_dev_directories("../dev_paths.txt")

In [ ]:
with open("../config/default_invasion_depth_computation.json", 'r') as fp:
    config = json.load(fp)

In [ ]:
config

In [ ]:
with open("../model_training/invasion_depth_training_values.json", 'r') as fp:
    training_values = json.load(fp)
training_values["rs_seed"] = None if (training_values["rs_seed"] == "None") else training_values["rs_seed"]

In [ ]:
training_values

In [ ]:
with open("../model_training/invasion_depth_best_hp.json", 'r') as fp:
    best_hp = json.load(fp)

In [ ]:
best_hp

In [ ]:
zstacks_root_path = f"{dirs.data_dir}/invasion_data/demo"

In [ ]:
### Data paths ###
model_training_path = f"{dirs.analysis_dir}/resnet50_invasion_model"
best_ensemble_training_path = f"{model_training_path}/best_ensemble"
inv_depth_output_path = f"{dirs.analysis_dir}/invasion_depth_output"

# Binary classification -> only need 1 output unit
n_outputs = 1
n_models = training_values["n_models"]
n_pred_models = config["n_pred_models"]

cls_thresh = training_values["cls_thresh"]
resnet_inp_shape = tuple(training_values["resnet_inp_shape"])
class_labels = training_values["class_labels"]
last_resnet_layer = best_hp["last_resnet_layer"]
# Zstack index meaning (descending: 0 -> bottom of well, ascending: 0 -> top of well)
descending = True

In [ ]:
data_prep.make_dir(inv_depth_output_path)

In [ ]:
zstack_dirs = [fp.replace("\\", "/") for fp in glob(f"{zstacks_root_path}/*")]

In [ ]:
zstack_dirs

# Build models

In [ ]:
best_val_losses = np.zeros(n_models)
for i in range(n_models):
    h_df = pd.read_csv(f"../model_training/best_ensemble/best_model_history_{i}.csv")
    ft_h_df = h_df.query("training_stage=='finetune'")
    best_val_losses[i] = ft_h_df.val_loss.min()

In [ ]:
sorted_best_model_idx = best_val_losses.argsort()

In [ ]:
# Load n_pred_models best models
K.clear_session()
inv_depth_models = [
    models.build_ResNet50_TL(n_outputs, resnet_inp_shape, base_last_layer=last_resnet_layer, base_model_trainable=False) for _ in range(n_pred_models)
]

In [ ]:
sorted_best_model_idx

In [ ]:
for i, m in enumerate(inv_depth_models):
    ith_best_idx = sorted_best_model_idx[i]
    m.trainable = True
    m.load_weights(f"../model_training/best_ensemble/best_finetune_weights_{ith_best_idx}.h5")
    m.trainable = False

# Make predictions

In [ ]:
for zstack_dir in zstack_dirs:
    ### Load data ###
    zpaths = zs.zstack_paths_from_dir(zstack_dir, descending=descending)
    x = data_prep.prep_inv_depth_imgs(zpaths, resnet_inp_shape[:-1])
    # Convert to tensor before calling predict() to speed up execution
    x = tf.convert_to_tensor(x, dtype="float")

    
    ### Make predictions ###
    # Probability predictions of each model
    yhatp_m = np.array(
        d.compute([d.delayed(m.predict)(x).squeeze() for m in inv_depth_models])[0]
    ).T
    # Mean probability predictions (ensemble predictions)
    yhatp = np.mean(yhatp_m, axis=1, keepdims=True)
    # Threshold probability predictions
    yhat = (yhatp > cls_thresh).astype(np.int32)

    
    ### Save outputs ###
    zstack_id = zstack_dir.split("/")[-1]
    output_file = pd.DataFrame({"img_name": [zp.split("/")[-1] for zp in zpaths], "inv_prob": yhatp.squeeze(), "inv_label": yhat.squeeze()})
    output_file.to_csv(f"{inv_depth_output_path}/{zstack_id}.csv", index=False)